# Majority voting to get final Deep Medic predictions

In [1]:
from MEDIcaTe.file_folder_ops import *
from MEDIcaTe.majority_vote import majority_vote
from MEDIcaTe.nii_resampling import *
import torchio as tio

/homes/kovacs/anaconda3/envs/nn_unet_cnl1/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# paths to the prediction of each Deep Medic model
pred0 = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/output/output_HNC_r196/predictions/testSessionDm_r196_m0/predictions'
pred1 = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/output/output_HNC_r196/predictions/testSessionDm_r196_m1/predictions'
pred2 = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/output/output_HNC_r196/predictions/testSessionDm_r196_m2/predictions'
pred3 = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/output/output_HNC_r196/predictions/testSessionDm_r196_m3/predictions'
pred4 = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/output/output_HNC_r196/predictions/testSessionDm_r196_m4/predictions'

# path to ground truth cases
gt = '/homes/kovacs/project_data/hnc-auto-contouring/hnc_rigs_all/data_nifty/t_r196/labels'

# destination of majority voted result
mv_dest = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/output_pred_majority_voted'
mv_dest_erroneous_name_fixed = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/output_pred_majority_voted_erroneous_name_fixed'

# destination of majority voted result resampled to original contouring space of ground truth
result_dest = '/homes/kovacs/project_data/hnc-auto-contouring/hnc_rigs_all/data_nifty/t_r196/predictions/deep_medic_norm/labels_deep_medic_predict_norm'

### Do majority voting
Simply add the 5 different predictions and keep as delineation if more than half say voxel is tumor. 

In [3]:
for i,case in enumerate(listdir(gt)):
    if i > 4:
        break
    pred_f_name = f'{case[:-7]}_pred_Segm.nii.gz'
    pred0_abspath = join(pred0,pred_f_name)
    pred1_abspath = join(pred1,pred_f_name)
    pred2_abspath = join(pred2,pred_f_name)
    pred3_abspath = join(pred3,pred_f_name)
    pred4_abspath = join(pred4,pred_f_name)
    out_abs_fname = join(mv_dest,case)
    print(pred_f_name)
    print(pred0_abspath)
    print(out_abs_fname)
    #if not isfile(out_abs_fname):
    print('doing majority voting')
    majority_vote(out_abs_fname,pred0_abspath,pred1_abspath,pred2_abspath,pred3_abspath,pred4_abspath)


HNC02_092_pred_Segm.nii.gz
/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/output/output_HNC_r196/predictions/testSessionDm_r196_m0/predictions/HNC02_092_pred_Segm.nii.gz
/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/output_pred_majority_voted/HNC02_092.nii.gz
doing majority voting
5
(512, 512, 175)
[0. 1. 2. 3. 4. 5.]
(512, 512, 175)
[0. 1.]
17481.0


In [7]:
# resample inferred volumes to original size

for i,case in enumerate(listdir(gt)):
    orig_f_name = join(gt,case)
    pred_f_name = join(mv_dest_erroneous_name_fixed,case)
    
    #lab_orig = tio.LabelMap(orig_f_name)
    lab_pred = tio.LabelMap(pred_f_name)
    resample_transform = tio.Resample(orig_f_name)
    resampled = resample_transform(lab_pred)

    outname=join(result_dest,case)
    resampled.save(outname)
    print(f'saved {outname}')


saved /homes/kovacs/project_data/hnc-auto-contouring/hnc_rigs_all/data_nifty/t_r196/predictions/deep_medic_norm/labels_deep_medic_predict_norm/HNC02_000.nii.gz
saved /homes/kovacs/project_data/hnc-auto-contouring/hnc_rigs_all/data_nifty/t_r196/predictions/deep_medic_norm/labels_deep_medic_predict_norm/HNC02_001.nii.gz
saved /homes/kovacs/project_data/hnc-auto-contouring/hnc_rigs_all/data_nifty/t_r196/predictions/deep_medic_norm/labels_deep_medic_predict_norm/HNC02_002.nii.gz
saved /homes/kovacs/project_data/hnc-auto-contouring/hnc_rigs_all/data_nifty/t_r196/predictions/deep_medic_norm/labels_deep_medic_predict_norm/HNC02_003.nii.gz
saved /homes/kovacs/project_data/hnc-auto-contouring/hnc_rigs_all/data_nifty/t_r196/predictions/deep_medic_norm/labels_deep_medic_predict_norm/HNC02_004.nii.gz
saved /homes/kovacs/project_data/hnc-auto-contouring/hnc_rigs_all/data_nifty/t_r196/predictions/deep_medic_norm/labels_deep_medic_predict_norm/HNC02_005.nii.gz
saved /homes/kovacs/project_data/hnc-aut

In [ ]:
#TODO: NEXT STEOP IS GENERATING THE METRICS

#visualize a case. look ok?
from MEDIcaTe.visualize_labels_ct_pet import visualization
final_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/output/output_HNC_r196/majority_voted/viualize'
ct='/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/t_r196_raw_data_base/images/HNC02_001_0000.nii.gz'
pet='/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/t_r196_raw_data_base/images/HNC02_001_0001.nii.gz'
l1='/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/t_r196_raw_data_base/labels/HNC02_001.nii.gz'
l2='/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/output/output_HNC_r196/majority_voted/testSessionDm_r196/HNC02_001_pred_mv.nii.gz'
visualization(final_path,ct_path=ct,pet_path=pet,label1_path=l1,label2_path=l2)